In [21]:
import pandas
import numpy
import tqdm
import datetime, time
from itertools import chain
import collections, functools, operator
import warnings


warnings.filterwarnings('ignore')

path = "C:/Users/Lenovo/Desktop/FCS/"
n = 10000
n = 0

class Main :
    def reader(self, fname, nrows, com='read_csv', sep=','):
        self.fname = fname
        self.nrow = nrows
        x = {}
        if nrows > 0:
            exe = "import pandas ; f = pandas." + com + "('" + fname + "'," + "sep='" + sep + "', nrows=" + str(nrows) + ")"
        else:
            exe = "import pandas ; f = pandas." + com + "('" + fname + "'," + "sep='" + sep + "')"
        exec(exe, x)
        return x['f']


Mn = Main()

In [3]:
### POUNTO 1
fname_ll = path + "loans_lenders.csv"
loans_lender = Mn.reader(fname=fname_ll, nrows=n); del fname_ll
loans_lender_N = pandas.DataFrame({"loan_id": numpy.repeat(loans_lender["loan_id"],
                                                           loans_lender["lenders"].str.split(',').map(len)),
                                   "lenders": list(chain.from_iterable(loans_lender["lenders"].str.split(', ')))})
print(loans_lender_N.head())

   loan_id      lenders
0   483693       muc888
0   483693      sam4326
0   483693  camaran3922
0   483693  lachheb1865
0   483693  rebecca3499


In [22]:
### PUNTO 2
fname_l = path + "loans.csv"
loans_ = Mn.reader(fname=fname_l, nrows=n); del fname_l

dur = loans_.iloc[:, [0, 19, 20]]
dur["planned_expiration_time"] = pandas.to_datetime(dur["planned_expiration_time"])
dur["disburse_time"] = pandas.to_datetime(dur["disburse_time"])

dur.insert(3, "loans_duration", (dur.iloc[:, 1]-dur.iloc[:, 2]).astype('timedelta64[D]'), True)
dur["loans_duration"].apply(lambda i: int(i) if pandas.notnull(i) else i)

loans_lender_ND = pandas.merge(loans_lender_N, dur.iloc[:, [0, 3]], on="loan_id"); del dur
print(loans_lender_ND.head())

   loan_id      lenders  loans_duration
0   483693       muc888            57.0
1   483693      sam4326            57.0
2   483693  camaran3922            57.0
3   483693  lachheb1865            57.0
4   483693  rebecca3499            57.0


In [8]:
### PUNTO 3
m = pandas.merge(loans_lender_N, loans_[["loan_id", "repayment_interval"]][loans_["repayment_interval"] != "bullet"],
                 on="loan_id")
list_of_lenders_that_have_funded_at_least_twice = list(dict.fromkeys(m["lenders"])); del m
print(list_of_lenders_that_have_funded_at_least_twice[0:9])

['muc888', 'sam4326', 'camaran3922', 'lachheb1865', 'rebecca3499', 'karlheinz4543', 'jerrydb', 'paula8951', 'gmct']


In [10]:
### PUNTO 4
list_country = []

list_pos = [] # per punto 5

for el in tqdm.tqdm(range(0, len(loans_.index))):
    if type(loans_["town_name"][el]) == float and type(loans_["borrower_genders"][el]) == float:
        list_country.append(loans_["country_name"][el])
        list_pos.append(el)
del el
count_country = pandas.DataFrame.from_dict({i: list_country.count(i) for i in list_country},
                                           orient="index", columns=["loans"])
print(count_country.iloc[0:10,])

100%|█████████████████████████████████████████████████████████████████████| 1419607/1419607 [00:46<00:00, 30305.85it/s]


             loans
Togo          2005
Cambodia       493
Kenya        10570
Rwanda         577
El Salvador   2058
Afghanistan    653
Tajikistan     311
Nicaragua     1140
Peru          2062
Palestine      112


In [11]:
### PUNTO 5
country_loan_d = {i: 0 for i in list_country}
for el in tqdm.tqdm(list_pos):
    if loans_["country_name"][el] in list_country:
        country_loan_d[loans_["country_name"][el]] += loans_["loan_amount"][el]
del el
country_loan = pandas.DataFrame.from_dict(country_loan_d, orient="index", columns=["loans"])
print(country_loan.iloc[0:10,])

100%|█████████████████████████████████████████████████████████████████████████| 43251/43251 [00:03<00:00, 12552.45it/s]


                 loans
Togo         1584875.0
Cambodia      346075.0
Kenya        4957750.0
Rwanda        940325.0
El Salvador  1241150.0
Afghanistan   624450.0
Tajikistan    373050.0
Nicaragua     717550.0
Peru         1462425.0
Palestine     204775.0


In [12]:
### PUNTO 6
total = loans_["loan_amount"].sum(axis=0, skipna=True)
l = list(country_loan_d.values())/total; l = l.tolist()
z = zip(list_country, l*100)
country_loan_prop = pandas.DataFrame.from_dict(dict(z), orient="index", columns=["loans"])
del l; del z; del list_country; del country_loan_d
print(country_loan_prop.iloc[0:10,])

                loans
Togo         0.000023
Cambodia     0.000010
Kenya        0.000008
Rwanda       0.000003
El Salvador  0.000088
Afghanistan  0.000185
Tajikistan   0.000038
Nicaragua    0.000008
Peru         0.000042
Palestine    0.000088


In [13]:
### PUNTO 7
list_country_Y = []

for el in tqdm.tqdm(range(0, len(loans_.index))):
    if type(loans_["disburse_time"][el]) != float:
        if type(loans_["town_name"][el]) == float and type(loans_["borrower_genders"][el]) == float:
            list_country_Y.append(loans_["country_name"][el] + " " + loans_["disburse_time"][el][0:4])
del el
count_country_Y = pandas.DataFrame.from_dict({i: list_country_Y.count(i) for i in list_country_Y}, orient="index",
                                             columns=["loans"])

country_loan_Y_d = {i: 0 for i in list_country_Y}
for el in tqdm.tqdm(list_pos):
    if type(loans_["disburse_time"][el]) != float:
        if loans_["country_name"][el] + " " + loans_["disburse_time"][el][0:4] in list_country_Y:
            country_loan_Y_d[loans_["country_name"][el] +" "+ loans_["disburse_time"][el][0:4]] += loans_["loan_amount"][el]
del el
country_loan_Y = pandas.DataFrame.from_dict(country_loan_Y_d, orient="index", columns=["loans"])

l = list(country_loan_Y_d.values())/total; l = l.tolist(); del total
z = zip(list_country_Y, l)
country_loan_prop_Y = pandas.DataFrame.from_dict(dict(z), orient="index", columns=["loans_%"])
del l; del z; del list_country_Y; del country_loan_Y_d
print(country_loan_prop_Y.iloc[0:20,])

100%|██████████████████████████████████████████████████████████████████████████| 43251/43251 [00:07<00:00, 5768.91it/s]


                                            loans_%
Togo 2012                              1.504100e-04
Cambodia 2015                          9.431309e-05
Cambodia 2016                          4.232133e-06
Kenya 2014                             1.005132e-05
Rwanda 2016                            1.779824e-04
Kenya 2016                             1.629371e-06
El Salvador 2015                       4.204624e-05
El Salvador 2012                       1.067132e-04
Afghanistan 2010                       5.685871e-05
Tajikistan 2014                        4.924087e-05
Nicaragua 2015                         2.108660e-04
Kenya 2015                             2.539280e-05
Peru 2012                              6.348200e-07
Palestine 2013                         1.239169e-04
Sierra Leone 2012                      3.605777e-05
Mexico 2014                            9.649264e-06
Philippines 2016                       2.835529e-06
Colombia 2012                          5.178015e-05
Lao People's

In [15]:
### PUNTO 8
m = pandas.merge(loans_lender, loans_[["loan_id", "loan_amount"]], on="loan_id")
m1 = pandas.DataFrame({"loan_id": m["loan_id"].astype(int),
                       "loan_amount_part": m["loan_amount"]/loans_lender["lenders"].str.split(',').map(len)}); del m
m2 = pandas.merge(loans_lender_N, m1, on="loan_id"); del m1
overall_amount_of_money_lent_per_lender = m2[["lenders", "loan_amount_part"]].groupby(
    ["lenders"])["loan_amount_part"].sum().reset_index(); del m2
print(overall_amount_of_money_lent_per_lender.iloc[0:10,])

          lenders  loan_amount_part
0             000       4439.157794
1           00000       3377.806092
2            0002       2657.022974
3        00mike00         52.631579
4  0101craign0101       7705.846364
5         0132575         94.444444
6         0154884        107.534613
7         0161130         51.602328
8         0169713        124.829008
9         0185429         44.791667


In [23]:
### PUNTO 9
fname_landers = path + "lenders.csv"
loans_lenders_dt = Mn.reader(fname=fname_landers, nrows=n); del fname_landers

lenders_st = pandas.DataFrame({"lenders": loans_lenders_dt["permanent_name"],
                               "country_code": loans_lenders_dt["country_code"]})
diz = loans_lenders_dt.groupby("country_code").size()
proporzioni_debito = pandas.DataFrame({"prop":diz/sum(diz)}); del diz
m = pandas.merge(overall_amount_of_money_lent_per_lender, lenders_st, on="lenders")
df1 = pandas.DataFrame(m.groupby("country_code").sum())
df1 = df1.rename(columns={"loan_amount_part": "credits"})
count = 0
for el in tqdm.tqdm(range(0, len(m.index))):
    if type(m["country_code"][el]) == float:
        count += m["loan_amount_part"][el]
del m
df2 = pandas.DataFrame({"credits": count*proporzioni_debito["prop"]})
del count; del proporzioni_debito

df = df2.add(df1, fill_value=0)
del df1; del df2
df["country_code"] = df.index

fname_cs = path + "country_stats.csv"
loans_country_stat = Mn.reader(fname=fname_cs, nrows=n); del fname_cs

cl = country_loan
cl["country_name"] = cl.index
m = pandas.merge(cl, loans_country_stat[["country_code", "country_name", "population_below_poverty_line", "population"]],
                 on="country_name"); del cl
m = m.drop(columns=["country_name"])

difference_stp = pandas.merge(df, m, on="country_code")
del m; del df
difference_stp["diff"] = difference_stp["credits"]-difference_stp["loans"]
difference = difference_stp[["country_code", "diff"]]
print(difference.iloc[0:10,])

100%|█████████████████████████████████████████████████████████████████████| 1383799/1383799 [00:59<00:00, 23399.56it/s]


  country_code          diff
0           AF  7.111683e+04
1           AL -3.382695e+04
2           AM  1.393695e+05
3           AZ -4.078453e+05
4           BA  2.442948e+05
5           BF  1.164741e+04
6           BG  1.263782e+06
7           BI -3.870660e+05
8           BJ -3.848587e+05
9           BO -5.842768e+05


In [17]:
### PUNTO 10
difference_stp["LOP"] = difference_stp["diff"]/difference_stp["population"]
difference_on_population = difference_stp[["country_code", "LOP"]].sort_values(by=['LOP'], ascending=False)
print(difference_on_population.iloc[0:10,])

   country_code       LOP
71           US  8.840570
28           IL  0.640198
19           CR  0.594035
53           PR  0.519123
6            BG  0.178385
49           PA  0.129311
13           BZ  0.125643
15           CL  0.086769
64           TH  0.073441
4            BA  0.069659


In [19]:
### PUNTO 11
difference_on_population_without = difference_stp[["country_code", "LOP"]][
    difference_stp["population_below_poverty_line"] < 30].sort_values(by=['LOP'], ascending=False)
print(difference_on_population_without.iloc[0:10,])

   country_code       LOP
71           US  8.840570
28           IL  0.640198
19           CR  0.594035
6            BG  0.178385
49           PA  0.129311
15           CL  0.086769
64           TH  0.073441
4            BA  0.069659
75           ZA  0.059229
10           BR  0.056141


In [25]:
### PUNTO 12
ldd = loans_[["disburse_time", "planned_expiration_time", "loan_amount"]]
ldd = ldd.dropna()
for el in ["disburse_time", "planned_expiration_time"]:
    ldd[el] = ldd[el].astype("datetime64[ns]")
del el

ldd["yE"] = ldd["planned_expiration_time"].apply(lambda x: x.year)
ldd["yB"] = ldd["disburse_time"].apply(lambda x: x.year)
ldd["yd"] = ldd["yE"] - ldd["yB"]
ldd = ldd.drop(columns=["yE", "yB"])

ldd_1 = ldd[["disburse_time", "loan_amount"]][ldd["yd"] == 0]
ldd_1["year"] = ldd["disburse_time"].apply(lambda x: x.year)
ldd_1 = ldd_1.drop(columns=["disburse_time"])
df1 = pandas.DataFrame(ldd_1.groupby("year").sum())
df1 = df1.rename(columns={"loan_amount": "loan"})
df1["year"] = df1.index
del ldd_1

ldd_n = ldd[["disburse_time", "planned_expiration_time", "loan_amount", "yd"]][ldd["yd"] > 0]
ldd_n = ldd_n.reset_index(drop=True)
df2 = pandas.DataFrame(columns=["year", "loan"])
for i in tqdm.tqdm(range(0, len(ldd_n.index))):
    df2_c = pandas.DataFrame(columns=["year", "loan_T", "days"])
    ln = ldd_n["yd"][i]
    ln = ln.astype("int")
    for j in range(0, ln+1):
        year = ldd_n["disburse_time"][i].year + j
        if j == 0:
            days = datetime.datetime(ldd_n["disburse_time"][i].year+1, 1, 1) - ldd_n["disburse_time"][i]
            days = days.days
        elif j == ln:
            days = ldd_n["planned_expiration_time"][i] - datetime.datetime(ldd_n["planned_expiration_time"][i].year-1,
                                                                           12, 31)
            days = days.days
        else:
            days = 365
        df2_c = df2_c.append({"year": year, "loan_T": ldd_n["loan_amount"][i], "days": days}, ignore_index=True)
        df2_c["loan"] = df2_c["loan_T"]*df2_c["days"]/sum(df2_c["days"])
    df2_c = df2_c.drop(columns=["days", "loan_T"])
    df2 = pandas.concat([df2, df2_c], ignore_index=True)
del df2_c; del year; del ln; del ldd_n; del i; del j; del days

m = df1.append(df2, ignore_index=True)
del df1; del df2

total_amount_of_loans_Y = pandas.DataFrame(m.groupby("year").sum())
total_amount_of_loans_Y["year"] = total_amount_of_loans_Y.index
print(total_amount_of_loans_Y.iloc[:,0])

100%|█████████████████████████████████████████████████████████████████████████| 150611/150611 [19:05<00:00, 131.52it/s]


year
2011.0    6.299119e+05
2012.0    1.132750e+08
2013.0    1.249371e+08
2014.0    1.531511e+08
2015.0    1.597599e+08
2016.0    1.589876e+08
2017.0    1.718619e+08
2018.0    5.705071e+06
Name: loan, dtype: float64
